## Sample Module with Sample ModuleState

This notebook shows how to create a basic module that can be easily tested with the simulation

In [2]:
import attr

from simulation.module import Module, ModuleState
from simulation.state import State


@attr.s(kw_only=True)
class SampleState(ModuleState):
    variable_name: int = attr.ib(default=0)

class Sample(Module):
    name = 'sample'
    defaults = {
        'ex_int': '0',
        'ex_float': '0.0',
        'ex_bool': 'False',
        'ex_string': '',
    }
    StateClass = SampleState
    
    def initialize(self, state: State):
        # how to get different values from config.ini file
        state.sample.ex_int = self.config.getint('ex_int')
        state.sample.ex_float = self.config.getfloat('ex_float')
        state.sample.ex_bool = self.config.getboolean('ex_bool')
        state.sample.ex_string = self.config.get('ex_string')
        
    # see simulation.module to see various default functions whose behaviour can be overridden

In [3]:
# see run in simulation/cli.py
from math import ceil

from simulation.config import SimulationConfig
from simulation.solver import advance, finalize, initialize


# any config files in config_files should be hardcoded to be a full local file path
# edit config file referencing geometry.hdf5 to be the full local file path
config_files = []
config = SimulationConfig(*config_files)

config.add_module(Sample)

target_time = 5
total = ceil(target_time / config.getfloat('simulation', 'time_step'))

attr.set_run_validators(config.getboolean('simulation', 'validate'))

def get_time(x):
    if x is None:
        return '0'
    return '%.2f' % x.time

state = initialize(State.create(config))
states = advance(state, target_time)
for _state in states:
    pass

state = finalize(state)

# Note: all files outputted will be in the same folder this notebook unless specified like below
state.save('../simulation-final.pkl')